In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

import warnings
warnings.filterwarnings('ignore')

plt.rcParams['figure.figsize'] = [10, 5]


In [2]:
def calStability(raw, task_order, method):
    values = {k:[] for k in method}

    for t in task_order:
        rows = raw[raw["task_order"]==t]
        offline = rows[rows["method"]=="offline"]
        
        for m in method:                
            target = rows[rows["method"]==m]
            _m = target[target["task_index"]==1][["accuracy", "no_of_test", "no_of_correct_prediction"]]
            
            if m=="offline":
                continue
                
            
            _ideal = offline[offline["task_index"]==1]["accuracy"]
            val = float((_m["accuracy"]/float(_ideal)).sum()/len(_m))
            values[m].append(val)
    
    return values

In [3]:
def calPlasticity(raw, task_order, method):
    values = {k:[] for k in method}

    for t in task_order:
        rows = raw[raw["task_order"]==t]
        offline = rows[rows["method"]=="offline"]
        
        for m in method:
            _sum = 0.0
                
            target = rows[rows["method"]==m]
            train_session = target["train_session"].unique()
            
            if m=="offline":
                continue
    
            _m = target[target["train_session"]==(target["task_index"])][["accuracy", "no_of_test", "no_of_correct_prediction"]]
            
            _ideal = offline["accuracy"]
            val = _m["accuracy"].div(_ideal.values, axis=0).sum()/len(_m)
            values[m].append(val)
            
    
    return values

In [4]:
def calOverallAcc(raw, task_order, method):
    values = {k:[] for k in method}

    for t in task_order:
        rows = raw[raw["task_order"]==t]
        offline = rows[rows["method"]=="offline"]
        
        for m in method:
            if m=="offline":
                continue
                    
            _sum = 0.0
                
            target = rows[rows["method"]==m]
            
            task_index = target["task_index"].unique()
            train_session = target["train_session"].unique()
            
            _m = target[target["train_session"]==len(task_index)][["accuracy", "no_of_test", "no_of_correct_prediction"]]
            
            _ideal = offline["accuracy"]
            val = _m["accuracy"].div(_ideal.values, axis=0).sum()/len(_m)
            values[m].append(val)
            
    return values

In [5]:
all_values = {}
for d in ["HouseA", "CASAS", "PAMAP", "DSADS"]:
    dataset = d
    folder = "../../Results/"+dataset+"/exp_no_of_hidden/"
    
    raw = pd.read_csv(folder+"results.txt")
    raw.columns = [c.strip() for c in raw.columns]
    raw["train_session"] = pd.to_numeric(raw["train_session"], errors='coerce')
    
    cmd = raw["cmd"].unique()
    task_order = raw["task_order"].unique()
    method = raw["method"].unique()

    stability = []
    plasticity = []
    overallAcc = []

    for c in cmd:
       
        target = raw[raw["cmd"]==c]
        m = calStability(target, task_order, method)
        stability.append(m)
        
        m = calPlasticity(target, task_order, method)
        plasticity.append(m)
        
        m = calOverallAcc(target, task_order, method)
        overallAcc.append(m)
        
    all_values[d] = (stability, plasticity, overallAcc)
    print(d, "DONE")

HouseA DONE
CASAS DONE
PAMAP DONE
DSADS DONE


# Continual learning 

In [7]:
from scipy import stats
import copy
def printCLMetrics(db, cmd, mgan5000=False):
    
    (stability, plasticity, overallAcc) = copy.deepcopy(all_values[db])
    
    if mgan5000:
        (_stability, _plasticity, _overallAcc) = _all_values[db]
        for model in ["mp-gan", "mp-wgan"]:
            stability[cmd][model] = _stability[0][model]
            plasticity[cmd][model] = _plasticity[0][model]
            overallAcc[cmd][model] = _overallAcc[0][model]
    
    stability  = stability[cmd]
    plasticity = plasticity[cmd]
    overallAcc = overallAcc[cmd]    
    
    def p(metric, name):
        print("Metric: ", name)
        for m in metric:
            avg = np.mean(metric[m])
            err = stats.sem(metric[m])
            print("{0} {1:.3f} {2:.3f}".format(m, avg, err))
        print("=====================")
        print("")
        
    p(stability, "M base")
    p(plasticity, "M new")
    p(overallAcc, "M overall")

In [10]:
printCLMetrics("HouseA", 0)

Metric:  M base
offline nan nan
none 0.358 0.007
exact 1.027 0.010
mp-gan 0.881 0.041
mp-wgan 0.861 0.048
sg-cgan 0.938 0.046
sg-cwgan 0.801 0.045
lwf 0.510 0.056
ewc 0.378 0.021

Metric:  M new
offline nan nan
none 1.126 0.007
exact 1.067 0.007
mp-gan 1.040 0.020
mp-wgan 1.054 0.022
sg-cgan 1.000 0.025
sg-cwgan 1.072 0.015
lwf 1.074 0.021
ewc 1.126 0.007

Metric:  M overall
offline nan nan
none 0.388 0.007
exact 1.006 0.006
mp-gan 0.863 0.038
mp-wgan 0.849 0.034
sg-cgan 0.824 0.031
sg-cwgan 0.824 0.040
lwf 0.535 0.040
ewc 0.385 0.008



In [11]:
printCLMetrics("CASAS", 1)

Metric:  M base
offline nan nan
none 0.286 0.050
exact 1.226 0.164
mp-gan 1.045 0.168
mp-wgan 0.908 0.183
sg-cgan 0.935 0.180
sg-cwgan 0.918 0.147
lwf 0.468 0.116
ewc 0.283 0.049

Metric:  M new
offline nan nan
none 1.195 0.048
exact 1.173 0.049
mp-gan 0.997 0.068
mp-wgan 1.090 0.067
sg-cgan 1.110 0.064
sg-cwgan 1.127 0.066
lwf 1.143 0.069
ewc 1.203 0.046

Metric:  M overall
offline nan nan
none 0.231 0.023
exact 1.037 0.038
mp-gan 0.756 0.042
mp-wgan 0.796 0.048
sg-cgan 0.766 0.036
sg-cwgan 0.784 0.043
lwf 0.498 0.043
ewc 0.229 0.023



In [12]:
printCLMetrics("PAMAP", 2)

Metric:  M base
offline nan nan
none 0.269 0.020
exact 1.116 0.054
mp-gan 0.537 0.035
mp-wgan 0.309 0.024
sg-cgan 0.648 0.065
sg-cwgan 0.386 0.030
lwf 0.269 0.020
ewc 0.226 0.011

Metric:  M new
offline nan nan
none 1.176 0.034
exact 0.977 0.055
mp-gan 1.143 0.017
mp-wgan 1.189 0.033
sg-cgan 1.147 0.037
sg-cwgan 1.148 0.041
lwf 1.176 0.034
ewc 1.196 0.035

Metric:  M overall
offline nan nan
none 0.319 0.023
exact 0.972 0.056
mp-gan 0.635 0.036
mp-wgan 0.445 0.048
sg-cgan 0.746 0.047
sg-cwgan 0.491 0.025
lwf 0.319 0.023
ewc 0.221 0.005



In [13]:
printCLMetrics("DSADS", 2)

Metric:  M base
offline nan nan
none 0.420 0.059
exact 1.972 0.306
mp-gan 0.580 0.068
mp-wgan 0.480 0.067
sg-cgan 1.011 0.189
sg-cwgan 0.518 0.071
lwf 0.420 0.059
ewc 0.396 0.062

Metric:  M new
offline nan nan
none 1.440 0.137
exact 0.848 0.110
mp-gan 1.335 0.118
mp-wgan 1.440 0.138
sg-cgan 1.348 0.118
sg-cwgan 1.434 0.139
lwf 1.440 0.137
ewc 1.612 0.102

Metric:  M overall
offline nan nan
none 0.327 0.045
exact 0.833 0.112
mp-gan 0.636 0.052
mp-wgan 0.391 0.055
sg-cgan 0.755 0.046
sg-cwgan 0.440 0.063
lwf 0.327 0.045
ewc 0.277 0.027



In [14]:
# def calStability(raw, task_order, method):
#     values = {k:[] for k in method}

#     for t in task_order:
#         rows = raw[raw["task_order"]==t]
#         offline = rows[rows["method"]=="offline"]
#         print("TASK ORDER", t)
#         for m in method:                
#             target = rows[rows["method"]==m]
#             _m = target[target["task_index"]==1][["accuracy", "no_of_test", "no_of_correct_prediction"]]
            
#             if m=="offline":
#                 continue
                
            
#             _ideal = offline[offline["task_index"]==1]["accuracy"]
            
#             if m=="exact":
#                 kk = _m["accuracy"]/float(_ideal)
#                 print("ideal", float(_ideal))
#                 print("model", list(_m["accuracy"]))
#                 for k in kk:
#                     print(k, end=" ")
#                 print("")
                
#             val = float((_m["accuracy"]/float(_ideal)).sum()/len(_m))
#             values[m].append(val)
    
#     return values


# all_values = {}
# for d in ["DSADS"]:
#     dataset = d
#     folder = "../../Results/"+dataset+"/exp_no_of_hidden/"
    
#     raw = pd.read_csv(folder+"results.txt")
#     raw.columns = [c.strip() for c in raw.columns]
#     raw["train_session"] = pd.to_numeric(raw["train_session"], errors='coerce')
    
#     cmd = raw["cmd"].unique()
#     task_order = raw["task_order"].unique()
#     method = raw["method"].unique()

#     stability = []
#     plasticity = []
#     overallAcc = []

#     for c in cmd:
#         if c != 3:
#             stability.append(None)
#             plasticity.append(None)
#             overallAcc.append(None)
#             continue
            
#         target = raw[raw["cmd"]==c]
#         m = calStability(target, task_order, method)
#         stability.append(m)
        
#         m = calPlasticity(target, task_order, method)
#         plasticity.append(m)
        
#         m = calOverallAcc(target, task_order, method)
#         overallAcc.append(m)
        
#     all_values[d] = (stability, plasticity, overallAcc)
#     print(d, "DONE")
    
# m = selectModel("DSADS")
# printCLMetrics(m, 3)